# 1. 네이버 뉴스 Crawling with Naver Open API
- 네이버 개발자 가입 : 서비스 API > 검색 > [오픈 API 이용 신청]
- 검색 API 사용 신청 : Products > 검색 > [오픈 API 이용 신청]

https://developers.naver.com/main/

In [ ]:
# 네이버 검색 API 예제 -  블로그 검색 예제를 뉴스 검색 예제로 수정하여 실행
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request

client_id = "fectRRiU4hHDSgWp0PYL"
client_secret = "CRcxk6VH6p"

# 한글 검색어 안전하게 변환
encText = urllib.parse.quote("인공지능")

# url + query 생성
url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과

# request message 구성
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)

# request ->response 받아오기
response = urllib.request.urlopen(request)

# 받아온 결과가 정상인지 확인
rescode = response.getcode()
if(rescode==200):
    # 정상이면 데이터 읽어오기
    response_body = response.read()
    # 한글이 있으면 utf-8 decoding
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

# 2. urllib : URL 작업용 패키지 (built-in)
* import urllib
* HTTP, FTP, SMTP 등과 같은 프로토콜을 사용하여 URL을 열고 읽고 쓰는 기능 제공

* https://ctkim.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-urllib-%EB%AA%A8%EB%93%88
   - urllib.request : 다양한 방식으로 URL을 열고 데이터를 가져오는 기능 제공
   - urllib.response : urllib가 사용하는 응답 클래스
   - urllib.parse : URL 파싱, 인코딩, 디코딩
   - urllib.robotparser : robots.txt 파일의 parsing
  

## 2-1. urllib으로 Open API 사용하기
  - 특수문자 및 비ASCII코드를 url에 사용할 수 있도록 변경 : *string* = **urllib.parse.quote**(*string*)
  - 요청 메시지 생성
    - Request 객체 생성 : *request_object* = **urllib.request.Request**(*url*)
    - Request 객체 Header에 API 사용을 위한 정보 추가 : *request_object*.add_header(*key*, *val*)
  - API 요청 및 결과 받기
    - Request 객체로 url 열고 HTTPResponse 객체 받기 : *Response* = **urllib.request.urlopen**(*Request*)
    - urlopen 결과 상태 정보 얻기 : *Response*.**getcode()** (**status**)
    - Response 객체에서 요청 결과 얻기 : *Response*.**read()**
    - 결과에 한글이 표함된 경우 UTF-8로 디코팅 : *string*.**decode**('utf-8')

## Naver 검색 API 동작 방식
   - start부터 display 수만큼 검색 결과를 json으로 가져옴
   - 검색 결과는 json의 'items'에 들어 있음
   - 검색 결과 수는 'display'에 들어 있음

In [ ]:
# 네이버 검색 API 예제 기반으로 네이버 검색 API 동작 방식 확인 (검색 결과 모두 받기)
# rescode가 200인 동안 start를 증가시켜서 계속 검색 (테스트를 위해 start는 25이하로 제한)
import os
import sys
import urllib.request

client_id = "fectRRiU4hHDSgWp0PYL"
client_secret = "CRcxk6VH6p"

# 한글 검색어 안전하게 변환
encText = urllib.parse.quote("인공지능")

# url + query 생성
url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과
start = 1
display = 10
rescode = 200
while rescode == 200 and start < 30:
    # request message 구성
    new_url = url + f"&start={start}&display={display}"
    request = urllib.request.Request(new_url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    
    # request ->response 받아오기
    response = urllib.request.urlopen(request)
    
    # 받아온 결과가 정상인지 확인
    rescode = response.getcode()
    if(rescode==200):
        # 정상이면 데이터 읽어오기
        response_body = response.read()
        # 한글이 있으면 utf-8 decoding
        print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)
        
    start += display

# 3. json : JSON 데이터 파싱,생성 (built-in)
* import json
* JSON 문자열을 파이썬 객체로 변환 : *pyObject* = **json.loads**(*jsonString*)
* 파이썬 객체를 JSON 문자열로 변환 : *jsonString* = **json.dumps**(*pyObject*)
   - ensure_ascii=True : 한글일때는 False로 변경하기
   - indent=None : 가독성을 높이기 위해 숫자 지정하기
* JSON 파일을 파이썬 객체로 로딩 : *pyObject* = **json.load**()
* 파이썬 객체를 JSON 파일에 저장 : **json.dump**(*pyObject*, *fileObject*)

https://docs.python.org/3/library/json.html

# [ 프로그램 Workflow ]
1. 네이버 뉴스 API로 Crawling : **searchNaverNews()** -> 파이썬 json 데이터
   1. url 구성하기
   1. url 접속과 검색 요청하기
   1. url 요청결과를 응답 json으로 받기 (json.loads)
1. 응답데이터를 리스트에 저장 : **setNewsSearchResult** -> List
1. 응답데이터가 없을 때까지 반복
   1. 파이썬 예외처리 : try + except + else + finally
1. 리스트를 CSV 파일로 저장 : **saveSearchResult** (pd.to_csv)

In [1]:
# 검색 API 호출, 응답을 JSON 데이터로 return하는 함수 작성
# API 호출 결과에 문제가 있을 경우 exception으로 처리
# exception 발생 시 exception과 호출 url 확인
import urllib.request
import json

def searchNaverNews(keyword, start, display):

    client_id = "fectRRiU4hHDSgWp0PYL"
    client_secret = "CRcxk6VH6p"

    # 한글 검색어 안전하게 변환
    encText = urllib.parse.quote(keyword)

    # url + query 생성
    url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과

    # request message 구성
    new_url = url + f"&start={start}&display={display}"
    request = urllib.request.Request(new_url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    resultJSON = None
    try:
        # request ->response 받아오기
        response = urllib.request.urlopen(request)
        
        # 받아온 결과가 정상인지 확인
        rescode = response.getcode()
        if(rescode==200):
            # 정상이면 데이터 읽어오기
            response_body = response.read()
            # 한글이 있으면 utf-8 decoding
            resultJSON = json.loads(response_body.decode('utf-8'))
        else:
            print("Error Code:" + rescode)
    except Exception as e:
        print(e)
        print(f"Error : {new_url}")

    return resultJSON

In [2]:
#응답데이터를 리스트에 저장 (검색 결과는 json의 'items'에 들어 있음)
def setNewsSearchResult(resultAll, resultJSON):
    for result in resultJSON['items']:
        resultAll.append(result)

In [3]:
# JSON의 list를 dataframe으로 변환하여 csv 파일로 저장
def saveSearchResult_CSV(json_list, filename):
    import pandas as pd
    data_df = pd.DataFrame(json_list)
    data_df.to_csv(filename)
    print(f"{filename} SAVED")

In [4]:
# 검색어 입력
keyword = input("검색어 : ").strip()
#keyword = "인공지능"

# API 호출 결과 전송된 JSON이 없거나, 전송된 JSON에 검색 결과가 없을 때까지
# 검색 결과를 JSON의 list로 저장 후 검색 API 추가 호출

# 검색 결과를 저장할 list 초기화
resultAll = []

# 첫 검색 API 호출
start = 1
display = 10
resultJSON = searchNaverNews(keyword, start, display)

while (resultJSON != None) and (resultJSON['display'] > 0):
    # 응답데이터 정리하여 리스트 저장
    setNewsSearchResult(resultAll, resultJSON)
 
    # 다음 검색 API 호출을 위한 파라미터 조정
    start += resultJSON['display']
    
    # API 호출
    resultJSON = searchNaverNews(keyword, start, display)
    
    # API 호출 성공 여부 출력
    if resultJSON != None:
        print(f"{keyword} [{start}] : Search Request Success")
    else:
        print(f"{keyword} [{start}] : Error ~~~~")

# 리스트를 csv 파일로 저장
filename = f"./data/{keyword}_naver_news.csv"
saveSearchResult_CSV(resultAll, filename)

검색어 :  22


22 [10] : Search Request Success
22 [20] : Search Request Success
22 [30] : Search Request Success
22 [40] : Search Request Success
22 [50] : Search Request Success
22 [60] : Search Request Success
22 [70] : Search Request Success
22 [80] : Search Request Success
22 [90] : Search Request Success
22 [100] : Search Request Success
22 [110] : Search Request Success
22 [120] : Search Request Success
22 [130] : Search Request Success
22 [140] : Search Request Success
22 [150] : Search Request Success
22 [160] : Search Request Success
22 [170] : Search Request Success
22 [180] : Search Request Success
22 [190] : Search Request Success
22 [200] : Search Request Success
22 [210] : Search Request Success
22 [220] : Search Request Success
22 [230] : Search Request Success
22 [240] : Search Request Success
22 [250] : Search Request Success
22 [260] : Search Request Success
22 [270] : Search Request Success
22 [280] : Search Request Success
22 [290] : Search Request Success
22 [300] : Search Reque

In [5]:
# 저장한 파일을 dataframe으로 loading 해보기
import pandas as pd
filename= f"./data/{keyword}_naver_news.csv"
data_df = pd.read_csv(filename, index_col=0)
data_df.head()

,title,originallink,link,description,pubDate
0,"김동연, 산불 피해 확산에 &quot;국가 비상사태…우리 일처럼 적극 지원&quot;",https://www.newsworks.co.kr/news/articleView.h...,https://www.newsworks.co.kr/news/articleView.h...,"경기도는 이번 산불과 관련해 긴급 대응체계를 가동, 지난 <b>22</b>일 소방인...","Wed, 26 Mar 2025 13:50:00 +0900"
1,"LG엔솔-델타 일렉트로닉스, 美 주택용 ESS시장 공략",http://amenews.kr/news/view.php?idx=61291,http://amenews.kr/news/view.php?idx=61291,"한편, 글로벌 시장조사업체 포춘 비즈니스 인사이트에 따르면 미국 주택용 ESS 시장...","Wed, 26 Mar 2025 13:50:00 +0900"
2,‘보물섬’ 염장선X지영수 치밀한 수싸움 예고,https://www.bntnews.co.kr/article/view/bnt2025...,https://www.bntnews.co.kr/article/view/bnt2025...,"지난 <b>22</b>일 방송된 10회가 전국 13.1%, 순간 최고 14.7%의 ...","Wed, 26 Mar 2025 13:50:00 +0900"
3,래미안원베일리 '국평' 70억에 거래됐다...평당 2억원 돌파,https://www.ajunews.com/view/20250326133544904,https://www.ajunews.com/view/20250326133544904,가장 최근 거래는 지난달 20일과 <b>22</b>일 84.95㎡(3층)와 84.9...,"Wed, 26 Mar 2025 13:50:00 +0900"
4,"[영상] 삼성전자 이재용 회장, 헬기타고 중국 BYD 방문…왜?",https://news.kbs.co.kr/news/pc/view/view.do?nc...,https://n.news.naver.com/mnews/article/056/001...,<b>22</b>일에는 샤오미 레이쥔 회장을 만나 손을 맞잡고 샤오미의 전기차 SU...,"Wed, 26 Mar 2025 13:50:00 +0900"
